In [1]:
import pandas as pd
import numpy as np

# GSEA  

## Find pathways with hits in the leading edge

In [2]:
pathways = ['Reactome', 'KEGG_Legacy', 'Wiki', 'GO']
entrez_hgnc_map = pd.read_csv('/home/upamanyu/GWANN/Code_AD/results_Sens8_v4/entrez_glist.csv', dtype=str)
entrez_hgnc_map = entrez_hgnc_map.set_index('entrezgene_id')['hgnc_symbol'].to_dict()

In [3]:
pw_dfs = {}
for pw in pathways:
    pw_df = pd.read_csv(f'/home/upamanyu/GWANN/Code_AD/results_Sens8_v4/enrichments/fGSEA_{pw}.csv', 
                        dtype={'hits_overlap': str})
    pw_df = pw_df.loc[~pw_df['hits_overlap'].isnull()]
    pw_df['hits'] = pw_df['hits_overlap'].apply(lambda x: ';'.join([entrez_hgnc_map[y] for y in x.split(';')])).values
    pw_df = pw_df[['pathway', 'padj', 'hits']]
    pw_df['hits_len'] = pw_df['hits'].apply(lambda x: len(x.split(';')))
    pw_df.sort_values(['hits_len', 'padj'], ascending=[False, True], inplace=True)
    pw_dfs[pw] = pw_df

In [4]:
pw_dfs['Wiki']

,pathway,padj,hits,hits_len
10,WP_VEGFA_VEGFR2_SIGNALING,0.003978,BIN1;ADAM10,2
7,WP_PLEURAL_MESOTHELIOMA,0.001681,ROR1,1
26,WP_PRIMARY_FOCAL_SEGMENTAL_GLOMERULOSCLEROSIS_...,0.043952,SYNPO,1


In [5]:
pw_dfs['Reactome']

,pathway,padj,hits,hits_len
4,Signaling by Receptor Tyrosine Kinases,0.000764,APOE;NRG3;APH1B;ADAM10,4
9,Diseases of signal transduction by growth fact...,0.011135,NRG3;APH1B;ADAM10,3
7,Axon guidance,0.002231,APH1B;ADAM10,2
1,Extracellular matrix organization,0.000104,ADAM10,1
33,Transcriptional regulation by RUNX1,0.029917,SPI1,1
37,PI3K events in ERBB2 signaling,0.035378,NRG3,1
43,PI3K events in ERBB4 signaling,0.037119,NRG3,1


In [6]:
pw_dfs['KEGG_Legacy']

,pathway,padj,hits,hits_len
8,KEGG_PATHWAYS_IN_CANCER,0.019593,SPI1,1
9,KEGG_ERBB_SIGNALING_PATHWAY,0.022878,NRG3,1


In [7]:
pw_dfs['GO'].head(10)


,pathway,padj,hits,hits_len
12,GOBP_SYNAPSE_ORGANIZATION,5.661460e-06,APOE;NRG3;LINGO2;ADAM10;SYNPO,5
3,GOCC_CELL_CELL_JUNCTION,7.035276e-08,LRRC7;PCDH9;ADAM10;SYNPO,4
9,GOCC_GLUTAMATERGIC_SYNAPSE,2.643126e-06,APOE;BIN1;NRG3;ADAM10,4
322,GOBP_REGULATION_OF_VESICLE_MEDIATED_TRANSPORT,4.769016e-02,APOE;BIN1;SORL1;SPI1,4
2,GOBP_REGULATION_OF_NEURON_PROJECTION_DEVELOPMENT,7.035276e-08,APOE;LRRC7;ROR1,3
10,GOCC_POSTSYNAPTIC_SPECIALIZATION,3.229511e-06,LRRC7;ADAM10;SYNPO,3
11,GOCC_NEURON_TO_NEURON_SYNAPSE,5.622494e-06,LRRC7;ADAM10;SYNPO,3
25,GOCC_DISTAL_AXON,7.770231e-05,BIN1;ROR1;PCDH9,3
29,GOCC_CELL_BODY,1.267556e-04,APOE;SORL1;SYNPO,3
52,GOBP_REGULATION_OF_SYNAPSE_STRUCTURE_OR_ACTIVITY,7.725888e-04,APOE;LINGO2;ADAM10,3


# Transcriptomic DEG enrichment (Patel et. al. studies)

## Combine DEG dataframes

In [2]:
# Find genes differentially expressed between AD and controls
patel1_ad_df = pd.read_csv('/mnt/sdb/Patel1/AD_DEG.csv', dtype={'Entrez_Gene_ID':int})
patel1_ad_df = patel1_ad_df[['Entrez_Gene_ID', 'Gene_Symbol', 'AW_FDR_adjusted_p_val', 'region']]
patel1_ad_df = patel1_ad_df.loc[patel1_ad_df['AW_FDR_adjusted_p_val'] < 0.05]
patel1_ad_df = patel1_ad_df[['Entrez_Gene_ID', 'Gene_Symbol', 'region']]
patel1_ad_df.drop_duplicates(inplace=True)
patel1_ad_df['Entrez_Gene_ID'] = patel1_ad_df['Entrez_Gene_ID'].astype(str)
patel1_ad_df.groupby('region')['Entrez_Gene_ID'].count()

region
cerebellum     867
frontal        460
parietal      1736
temporal       323
Name: Entrez_Gene_ID, dtype: int64

In [3]:
# Find genes differentially expressed between (i) AD and controls or 
# (ii) AsymAD and controls
patel2_ad_df = pd.read_csv('/mnt/sdb/Patel2/DEG.csv', dtype={'Entrez_Gene_ID':int})
patel2_ad_df = patel2_ad_df[['Entrez_Gene_ID', 'Gene_Symbol', 'AsymAD vs CO.P.val', 'AD vs CO.P.val', 'region']]
patel2_ad_df = patel2_ad_df.loc[(patel2_ad_df['AsymAD vs CO.P.val'] < 0.05) | 
                                (patel2_ad_df['AD vs CO.P.val'] < 0.05)]
patel2_ad_df = patel2_ad_df[['Entrez_Gene_ID', 'Gene_Symbol', 'region']]
patel2_ad_df.drop_duplicates(inplace=True)
patel2_ad_df['Entrez_Gene_ID'] = patel2_ad_df['Entrez_Gene_ID'].astype(str)
patel2_ad_df.groupby('region')['Entrez_Gene_ID'].count()


region
cerebellum     176
entorhinal    1695
frontal        556
temporal      1625
Name: Entrez_Gene_ID, dtype: int64

## Convert DEG dicts to GMT files to use with fGSEA

In [4]:
patel1_dict = patel1_ad_df.groupby('region')['Entrez_Gene_ID'].apply(list).to_dict()
patel1_dict = {f'Patel1_{k}': v for k, v in patel1_dict.items()}
patel2_dict = patel2_ad_df.groupby('region')['Entrez_Gene_ID'].apply(list).to_dict()
patel2_dict = {f'Patel2_{k}': v for k, v in patel2_dict.items()}
deg_dict = {**patel1_dict, **patel2_dict}
gmt_text = ''
for k, v in deg_dict.items():
    gmt_text += f'{k.upper()}\t' + 'no_website\t' + '\t'.join(v) + '\n'
with open('Patel_DEG.entrez.gmt', 'w') as f:
    f.write(gmt_text)

## Intersection over union of DEGs between two studies

In [30]:
print(f'{"Region":<10}   {"AuB":<10}{"A^B":<10}{"A-B":<10}{"B-A":<10}\n')
for region in ['temporal', 'frontal', 'cerebellum']:
    aunib = len(set(deg_dict[f'Patel1_{region}']).union(set(deg_dict[f'Patel2_{region}'])))
    aintb = len(set(deg_dict[f'Patel1_{region}']).intersection(set(deg_dict[f'Patel2_{region}'])))
    adiffb = len(set(deg_dict[f'Patel1_{region}']).difference(set(deg_dict[f'Patel2_{region}'])))
    bdiffa = len(set(deg_dict[f'Patel2_{region}']).difference(set(deg_dict[f'Patel1_{region}'])))
    print(f'{region:<10} : {aunib:<10}{aintb:<10}{adiffb:<10}{bdiffa:<10}')
    print(f'{"":<10} : {"":<10}{aintb/aunib:.4f}    {adiffb/aunib:.4f}    {bdiffa/aunib:.4f}')
    

Region       AuB       A^B       A-B       B-A       

temporal   : 1782      166       157       1459      
           :           0.0932    0.0881    0.8187
frontal    : 921       95        365       461       
           :           0.1031    0.3963    0.5005
cerebellum : 954       89        778       87        
           :           0.0933    0.8155    0.0912


## fGSEA results to table

In [25]:
fgsea_res = pd.read_csv('/home/upamanyu/GWANN/Code_AD/results_Sens8_v4/enrichments/fGSEA_Patel_DEG.csv')
fgsea_res = fgsea_res[['pathway', 'padj', 'hits_overlap']]
fgsea_res['Study'] = fgsea_res['pathway'].apply(lambda x: x.split('_')[0].title())
fgsea_res['Brain Region'] = fgsea_res['pathway'].apply(lambda x: x.split('_')[1].title())
fgsea_res.rename(columns={'padj': 'P-value'}, inplace=True)
fgsea_res = fgsea_res[['Study', 'Brain Region', 'P-value', 'hits_overlap']]
fgsea_res
res_table = fgsea_res.groupby(['Study', 'Brain Region'])[['P-value', 'hits_overlap']].first().stack().unstack(1).T
res_table.to_excel('/home/upamanyu/GWANN/Code_AD/results_Sens8_v4/enrichments/fGSEA_Patel_DEG.xlsx')
res_table

Study           Patel1                   Patel2                
               P-value   hits_overlap   P-value    hits_overlap
Brain Region                                                   
Cerebellum    0.038628           5101  0.678322             NaN
Entorhinal         NaN            NaN  0.002583  274;6653;11346
Frontal       0.009741           6653  0.388183             348
Parietal           0.0  348;6653;5101       NaN             NaN
Temporal      0.009028            NaN  0.031962           11346